# グリッドサーチ（Grid Search）
あらかじめ設定した各パラメータの候補全ての組み合わせを試す方法。  
「最適なパラメータ組み合わせ」が得られる  
探索空間が大きいと計算コストが上がるので、事前に候補を絞る必要がある

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# シード固定
torch.manual_seed(42)
np.random.seed(42)

# サンプルデータ生成：y = 2*x + 1 にノイズを加えた回帰データ
X = np.linspace(-1, 1, 100).reshape(-1, 1)
y = 2 * X + 1 + np.random.normal(0, 0.2, X.shape)

# NumPy → PyTorch tensor
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# シンプルなMLPモデル（1層の隠れ層）
class SimpleMLP(nn.Module):
    def __init__(self, hidden_size):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(1, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# モデル訓練関数：指定したエポック数だけ訓練し、最終MSEを返す
def train_model(hidden_size, lr, epochs=100):
    model = SimpleMLP(hidden_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(X_tensor)
        loss = criterion(output, y_tensor)
        loss.backward()
        optimizer.step()
    return loss.item()


In [8]:
# 探索候補（例：隠れ層のサイズと学習率）
hidden_sizes = [4, 8, 16]
learning_rates = [0.001, 0.01, 0.1]

best_loss = float('inf')
best_params = None

for hidden in hidden_sizes:
    for lr in learning_rates:
        loss = train_model(hidden, lr, epochs=100)
        print(f"Hidden: {hidden}, LR: {lr} -> Loss: {loss:.4f}")
        if loss < best_loss:
            best_loss = loss
            best_params = (hidden, lr)

print("Best parameters (Grid Search):", best_params, "with Loss:", best_loss)


Hidden: 4, LR: 0.001 -> Loss: 1.2330
Hidden: 4, LR: 0.01 -> Loss: 0.1652
Hidden: 4, LR: 0.1 -> Loss: 0.0321
Hidden: 8, LR: 0.001 -> Loss: 1.8177
Hidden: 8, LR: 0.01 -> Loss: 0.0349
Hidden: 8, LR: 0.1 -> Loss: 0.0319
Hidden: 16, LR: 0.001 -> Loss: 1.1691
Hidden: 16, LR: 0.01 -> Loss: 0.0320
Hidden: 16, LR: 0.1 -> Loss: 0.0302
Best parameters (Grid Search): (16, 0.1) with Loss: 0.030203668400645256
